In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import warnings
import os.path
import info
#import lightgbm as lgb
from utils import *

warnings.filterwarnings("ignore")

path_pc = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/'
today = datetime.today()
print("Today's date:", today.strftime("%Y-%m-%d"))

Today's date: 2020-04-06


In [2]:
outcomes_new = load_latest(today, 'outcomes_', path_pc)

Trying  outcomes_2020-04-06.csv
Loading file:  outcomes_2020-04-06.csv


In [3]:
master_scoreboard = load_latest(today, 'master_scoreboard_', path_pc)

Trying  master_scoreboard_2020-04-06.csv
Trying  master_scoreboard_2020-04-05.csv
Trying  master_scoreboard_2020-04-04.csv
Trying  master_scoreboard_2020-04-03.csv
Trying  master_scoreboard_2020-04-02.csv
Loading file:  master_scoreboard_2020-04-02.csv


In [4]:
top_5_total_gain_symbols = master_scoreboard.sort_values(by=['Test Total Gain'], ascending=False)['symbol'][0:5]

In [5]:
top_5_profit_percentage_symbols = master_scoreboard.sort_values(by=['Profit Percentage'], ascending=False)['symbol'][0:5]

In [6]:
top_5_gain_loss_ratio_symbols = master_scoreboard.sort_values(by=['Gain Loss Ratio'], ascending=False)['symbol'][0:5]

In [7]:
selected_symbols = set(top_5_total_gain_symbols).union(set(top_5_profit_percentage_symbols))

In [8]:
selected_symbols
#selected_symbols = {'0027.HK', '0941.HK', '1044.HK', '1093.HK', '2382.HK'}

{'0027.HK', '0700.HK', '1044.HK', '1088.HK', '1093.HK', '1109.HK', '1398.HK'}

In [9]:
outcomes_new.replace(0, np.nan, inplace=True)

In [10]:
outcomes_new

(close/ema)-1  (close/sma)-1  1-lower/close  \
date       symbol                                                 
2000-01-03 0005.HK            NaN            NaN            NaN   
2000-01-04 0001.HK            NaN            NaN            NaN   
           0002.HK            NaN            NaN            NaN   
           0003.HK            NaN            NaN            NaN   
           0005.HK      -0.009161      -0.009346       0.009434   
...                           ...            ...            ...   
2020-04-06 2382.HK      -0.063603      -0.089336       0.134801   
           2388.HK      -0.032226      -0.057887       0.141481   
           2628.HK      -0.088754      -0.109257       0.153880   
           3328.HK      -0.039300      -0.045065       0.057741   
           3988.HK      -0.020690      -0.025846       0.056355   

                    bbands50_lower  bbands50_lower_slope  bbands50_upper  \
date       symbol                                                          
2000-01-03 0005.HK      108.000000                   NaN      108.000000   
2000-01-04 0001.HK       70.042400                   NaN       70.042400   
           0002.HK       35.400000                   NaN       35.400000   
           0003.HK        3.329700                   NaN        3.329700   
           0005.HK      105.000000             -3.000000      109.000000   
...                            ...                   ...             ...   
2020-04-06 2382.HK       95.171916             -1.049176      146.410084   
           2388.HK       20.218113             -0.252571       29.775887   
           2628.HK       13.114861             -0.130020       21.687539   
           3328.HK        4.428619             -0.015158        5.414981   
           3988.HK        2.802625             -0.007292        3.294975   

                    bbands50_upper_slope     close  close_pct_change_5_day  \
date       symbol                                                            
2000-01-03 0005.HK                   NaN  108.0000                     NaN   
2000-01-04 0001.HK                   NaN   70.0424                     NaN   
           0002.HK                   NaN   35.4000                     NaN   
           0003.HK                   NaN    3.3297                     NaN   
           0005.HK              1.000000  106.0000                     NaN   
...                                  ...       ...                     ...   
2020-04-06 2382.HK             -0.402468  110.0000                0.079490   
           2388.HK             -0.005780   23.5500                0.116114   
           2628.HK             -0.163278   15.5000                0.038874   
           3328.HK             -0.017800    4.7000                0.012931   
           3988.HK             -0.007667    2.9700                0.024138   

                    close_scaled50  ...  past_log_return_3  past_log_return_4  \
date       symbol                   ...                                         
2000-01-03 0005.HK             NaN  ...                NaN                NaN   
2000-01-04 0001.HK             NaN  ...                NaN                NaN   
           0002.HK             NaN  ...                NaN                NaN   
           0003.HK             NaN  ...                NaN                NaN   
           0005.HK       -0.707107  ...                NaN                NaN   
...                            ...  ...                ...                ...   
2020-04-06 2382.HK       -0.833952  ...           0.053209           0.052251   
           2388.HK       -0.599494  ...           0.081814           0.093401   
           2628.HK       -0.878182  ...           0.032790           0.020861   
           3328.HK       -0.890428  ...           0.008547          -0.010582   
           3988.HK       -0.633760  ...           0.010152                NaN   

                    past_log_return_5  past_log_return_10  past_return_1  \
date       sym

In [11]:
outcomes_new = outcomes_new.drop(['log_return_10', 'return_10'#,
                                  #'log_return_5', 'return_5',
                                  #'log_return_4', 'return_4',
                                  #'log_return_3', 'return_3',
                                  #'log_return_2', 'return_2'
                                 ], axis=1)

In [12]:
features_selected = ['(close/ema)-1',
 '(close/sma)-1',
 '1-lower/close',
 #'bbands50_lower',
 'bbands50_lower_slope',
 #'bbands50_upper',
 'bbands50_upper_slope',
 #'close',
 'close_pct_change_5_day',
 'close_scaled50',
 'close_slope',
 #'ema50',
 'ema50_scaled50',
 'ema50_slope',
 #'high',
 'high_scaled50',
 'high_slope',
 'intraday_chg',
 'log volume',
 #'low',
 'low_scaled50',
 'low_slope',
 #'macd',
 #'macd_signal',
 'macd_signal_pct_diff',
 'momentum',
 #'open',
 'open_scaled50',
 'open_slope',
 'rsi14',
 'rsi14_slope',
 #'sma50',
 'sma50_scaled50',
 'sma50_slope',
 'upper/close-1',
 'volatility50',
 'volatility50_ratio',
 #'volume',
 'volume_pct_change_1_day',
 'volume_scaled50',
 'volume_slope',
 'past_return_1',
 'past_return_2',
 'past_return_3',
 'past_return_4',
 'past_return_5',
 'past_return_10',
 'past_log_return_1',
 'past_log_return_2',
 'past_log_return_3',
 'past_log_return_4',
 'past_log_return_5',
 'past_log_return_10'
 ]

In [13]:
last_date = outcomes_new.index.get_level_values('date')[-1].to_pydatetime()
print ("Last date in dataset: ", last_date.strftime("%Y-%m-%d"))

Last date in dataset:  2020-04-06


In [14]:
prob_threshold = 0.7
#test_period = 100
#test_size = 0
valid_size = 400
training_size = 2000
#valid_test_gap = 4
q_upper = 0.9
q_lower = 0.1
# return_col = 'log_return_1'
# return_col_actual = 'return_1'
#return_col = 'log_return_2'
#return_col_actual = 'return_2'
# return_col = 'log_return_3'
# return_col_actual = 'return_3'
return_col = 'log_return_5'
return_col_actual = 'return_5'
# return_col = 'log_return_10'
# return_col_actual = 'return_10'
max_depth_range = np.arange(2,5,1)
num_leaves_range = np.arange(9,15,1)

In [15]:
predict_data = outcomes_new[last_date:][features_selected]

In [16]:
outcomes_new_dropna = outcomes_new.dropna()

In [17]:
y_class_pred_all = pd.DataFrame()

for selected_symbol in selected_symbols:
    
    print("-"*80)
    print("\nWorking on ", selected_symbol)

    #select from dataset only the symbol we want
    outcomes_new_selected_symbol = outcomes_new_dropna.loc[outcomes_new_dropna.index.get_level_values('symbol')==selected_symbol]

    #get list of dates from dataset
    data_dates = list(set(outcomes_new_selected_symbol.index.get_level_values(0)))
    data_dates = sorted(data_dates)
    data_converted_dates = []
    for ts in data_dates:
        data_converted_dates.append(ts.to_pydatetime())
        
    if len(data_converted_dates) < valid_size+training_size:
        print("Skipped ", selected_symbol, " due to insuffient trading days.")
        continue

    #get dates for training and valid set
    start_date = data_converted_dates[-valid_size-training_size]
    start_date_valid = data_converted_dates[-valid_size-1]
    end_date_valid = data_converted_dates[-1]
    print("start training date: ", start_date)
    print("Valid dataset period: ", start_date_valid, " - ", end_date_valid)

    score_board = pd.DataFrame()
    
    print("Training for ", selected_symbol)

    #split into train, valid and test sets
    X_y_train, X_y_valid = train_valid_split(outcomes_new_selected_symbol, start_date,
                                                            start_date_valid, end_date_valid)

    #calculate upper threshold in training set, then create targets for both valid and test
    X_y_train, X_y_valid = add_target_upper_notest(X_y_train, X_y_valid, q_upper, 'target', return_col)

    #downsample the training set's negative data points
    X_y_train_resampled = downsample(X_y_train, 'target', test_ratio=0.11, random_seed=11)

    #create extra valid set for calculating gain
    X_valid_close = X_y_valid[['close',return_col_actual]]

    #split into features and target sets
    X_train, y_train = feature_target_split(X_y_train_resampled, features_selected, 'target')
    X_valid, y_valid = feature_target_split(X_y_valid, features_selected, 'target')
    
    X_test = predict_data.loc[predict_data.index.get_level_values('symbol') == selected_symbol]
    
    if X_test.isnull().values.any() or len(X_test)==0:
        print(selected_symbol, ' contains NaN value. Will skip prediction.')
        continue

    (best_model, best_pres_model, max_total_gain, optimal_depth, optimal_num_leaves, max_precision,
    optimal_precision_depth, optimal_precision_num_leaves,
    max_precision_total_gain) = lgb_train(X_train, y_train, X_valid, y_valid, X_valid_close, max_depth_range, num_leaves_range, return_col_actual,
                                         min_data = 11, metric = 'auc', prob_threshold = prob_threshold)

    print("\nMaking prediction for ", selected_symbol, " on ", last_date)
    
    y_test_pred = best_model.predict(X_test, num_iteration=best_model.best_iteration)
    y_class_pred = int(y_test_pred >= prob_threshold)
    y_class_pred_all = y_class_pred_all.append({'date': last_date,
                                                'symbol': selected_symbol,
                                                'pred':y_class_pred}, ignore_index=True)

--------------------------------------------------------------------------------

Working on  0700.HK
start training date:  2008-10-14 00:00:00
Valid dataset period:  2018-05-30 00:00:00  -  2020-03-30 00:00:00
Training for  0700.HK
upper_threshold:  0.06326368942049379

Making prediction for  0700.HK  on  2020-04-06 00:00:00
--------------------------------------------------------------------------------

Working on  1093.HK
start training date:  2005-07-22 00:00:00
Valid dataset period:  2018-03-27 00:00:00  -  2020-03-30 00:00:00
Training for  1093.HK
upper_threshold:  0.08245140465876748

Making prediction for  1093.HK  on  2020-04-06 00:00:00
--------------------------------------------------------------------------------

Working on  1109.HK
start training date:  2007-10-08 00:00:00
Valid dataset period:  2018-03-06 00:00:00  -  2020-03-30 00:00:00
Training for  1109.HK
upper_threshold:  0.0783975088130309

Making prediction for  1109.HK  on  2020-04-06 00:00:00
-----------------

In [18]:
X_test

,,(close/ema)-1,(close/sma)-1,1-lower/close,bbands50_lower_slope,bbands50_upper_slope,close_pct_change_5_day,close_scaled50,close_slope,ema50_scaled50,ema50_slope,...,past_return_3,past_return_4,past_return_5,past_return_10,past_log_return_1,past_log_return_2,past_log_return_3,past_log_return_4,past_log_return_5,past_log_return_10
date,symbol,,,,,,,,,,,,,,,,,,,,,
2020-04-06,1088.HK,0.006696,0.021344,0.08779,0.032581,-0.101806,-0.027285,0.618531,0.145802,-1.166891,-0.005137,...,0.002813,-0.033875,-0.027285,0.103715,0.008451,-0.022192,0.002809,-0.034462,-0.027664,0.098682


In [19]:
y_class_pred_all = y_class_pred_all.set_index(['date','symbol'])

In [20]:
y_class_pred_all

pred
date       symbol       
2020-04-06 0700.HK   0.0
           1093.HK   0.0
           1109.HK   1.0
           1044.HK   0.0
           0027.HK   0.0
           1088.HK   0.0

In [21]:
prediction_filename = 'predict_' + last_date.strftime("%Y-%m-%d") + '.csv'
print("Saving predictions to ", prediction_filename)
save_csv(y_class_pred_all, path_pc, prediction_filename)
print("Done predicting.")

Saving predictions to  predict_2020-04-06.csv
Done predicting.


In [22]:
print("\nLoading predictions...")
predict_date = get_last_date(outcomes_new)
prediction_filename = 'predict_' + predict_date.strftime("%Y-%m-%d") + '.csv'
y_class_pred = pd.read_csv(path_pc + prediction_filename, parse_dates = True, header=None)
y_class_pred.columns = ['date','symbol','pred']
y_class_pred = y_class_pred.drop(0)
y_class_pred['date'] = y_class_pred['date'].astype('datetime64[ns]')
y_class_pred = y_class_pred.set_index(['date','symbol'])


Loading predictions...


In [23]:
y_class_pred

pred
date       symbol      
2020-04-06 0700.HK  0.0
           1093.HK  0.0
           1109.HK  1.0
           1044.HK  0.0
           0027.HK  0.0
           1088.HK  0.0